#### Evaluation

In [ ]:
import json
import spacy
from sklearn.metrics import precision_score, recall_score, f1_score

# --- Loading spaCy model for aspect extraction ---
SPACY_MODEL_PATH = '/teamspace/studios/this_studio/Model/spacy/spacy_iteration_004'
spacy_nlp = spacy.load(SPACY_MODEL_PATH)
#------------------------------------------------
TEST_FILE = '/teamspace/studios/this_studio/test2.jsonl'
extracted_aspects_all = []
ground_truth_aspects_all = []

def calculate_iou(true_span, predicted_span):
    try:
        true_start, true_end = int(true_span[0]), int(true_span[1])
        pred_start, pred_end = int(predicted_span[0]), int(predicted_span[1])
    except ValueError:
        return 0.0  

    intersection_start = max(true_start, pred_start)
    intersection_end = min(true_end, pred_end)

    if intersection_start < intersection_end:
        intersection = intersection_end - intersection_start
    else:
        return 0.0
    
    union_start = min(true_start, pred_start)
    union_end = max(true_end, pred_end)
    union = union_end - union_start
    return intersection / union  

def evaluate_aspect_extraction(pred_aspects_all, gt_aspects_all):
    y_true_set = set()
    y_pred_set = set()
    iou_scores = []
    for idx, (preds, gts) in enumerate(zip(pred_aspects_all, gt_aspects_all)):
        for gt in gts:
            aspect, start, end = gt
            if start is not None and end is not None:
                y_true_set.add((idx, aspect, int(start), int(end)))
        for pred in preds:
            aspect, start, end = pred
            if start is not None and end is not None:
                y_pred_set.add((idx, aspect, int(start), int(end)))

    tp = len(y_true_set & y_pred_set)
    fp = len(y_pred_set - y_true_set)
    fn = len(y_true_set - y_pred_set)

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0.0

    for true_entity in y_true_set:
        for pred_entity in y_pred_set:
            if true_entity[1] == pred_entity[1]:  
                iou = calculate_iou(true_entity[2:], pred_entity[2:])
                iou_scores.append(iou)

    average_iou = sum(iou_scores) / len(iou_scores) if iou_scores else 0.0
    return precision, recall, f1, average_iou

with open(TEST_FILE, 'r') as f:
    for line in f:
        item = json.loads(line.strip())
        sentence = item['sentence']
        gt_entity = item['entities']
        gt_aspect = (gt_entity['aspect'], gt_entity['start_offset'], gt_entity['end_offset'])
        ground_truth_aspects_all.append([gt_aspect])
        doc = spacy_nlp(sentence)
        pred_aspects = []
        for ent in doc.ents:
            start = ent.start_char
            end = ent.end_char
            pred_aspects.append((ent.text, start, end))
        extracted_aspects_all.append(pred_aspects)


print("\n--- Aspect Extraction Evaluation ---\n")
precision, recall, f1, average_iou = evaluate_aspect_extraction(extracted_aspects_all, ground_truth_aspects_all)
print(f"Aspect Extraction Precision: {precision:.4f}")
print(f"Aspect Extraction Recall: {recall:.4f}")
print(f"Aspect Extraction F1: {f1:.4f}")
print(f"Average IoU: {average_iou:.4f}")

#### Visualization

In [ ]:
import matplotlib.pyplot as plt

iterations = ['1', '2', '3', '4']
precision_values = [0.1667, 0.4012, 0.4407, 0.5474]
recall_values = [0.0075, 0.5149, 0.5821, 0.5597]

plt.figure(figsize=(8, 5))
plt.plot(iterations, precision_values, marker='o', linestyle='-', color='red', linewidth=2, label='Precision')
plt.plot(iterations, recall_values, marker='o', linestyle='-', color='blue', linewidth=2, label='Recall')

for i, (p, r) in enumerate(zip(precision_values, recall_values)):
    plt.text(i, p + 0.02, f"{p:.4f}", ha='center', color='red')
    plt.text(i, r + 0.02, f"{r:.4f}", ha='center', color='blue')
    
plt.ylim(0, 1.1)
plt.xlabel("Iterations")
plt.ylabel("Score")
plt.title("Precision and Recall across iterations")
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

iterations = ['1', '2', '3', '4']
f1_values = [0.0143, 0.4510, 0.5016, 0.5535]
iou_values = [0.2500, 0.0618, 0.0630, 0.0730]

plt.figure(figsize=(8, 5))
plt.plot(iterations, f1_values, marker='o', linestyle='-', color='orange', linewidth=2, label='F1 Score')
plt.plot(iterations, iou_values, marker='o', linestyle='--', color='green', linewidth=2, label='IoU')
for i, (p, r) in enumerate(zip(f1_values, iou_values)):
    plt.text(i, p + 0.02, f"{p:.4f}", ha='center', color='orange')
    plt.text(i, r + 0.02, f"{r:.4f}", ha='center', color='green')

plt.ylim(0, 1.1)
plt.xlabel("Iterations")
plt.ylabel("Score")
plt.title("F1 score and IoU across Iterations")
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend()
plt.tight_layout()
plt.show()